In [1]:
!pip install semanticscholar python-dotenv requests -q

In [2]:
import json
import os
from semanticscholar import SemanticScholar
from dotenv import load_dotenv

In [3]:

def setup_api_key():
    """Set up API key either from .env file or directly"""
    # Method 1: Try loading from .env file
    load_dotenv()
    API_KEY = os.getenv("SEMANTIC_SCHOLAR_API_KEY")

    # Method 2: If not in .env, use direct key
    if not API_KEY:
        # Create .env file with your API key
        with open(".env", "w") as f:
            f.write("SEMANTIC_SCHOLAR_API_KEY=83rBkeaXb14D8vGpXJezU6nrCFFmyn5L8RCvT9MM\n")
        load_dotenv()
        API_KEY = os.getenv("SEMANTIC_SCHOLAR_API_KEY")

    # Initialize Semantic Scholar
    if API_KEY:
        sch = SemanticScholar(api_key=API_KEY)
        print("Semantic Scholar initialized with API key")
    else:
        sch = SemanticScholar()
        print(" Using Semantic Scholar without API key (limited rate)")

    return sch


In [4]:
def search_papers(topic, limit=20):
    """
    Search Semantic Scholar for papers on a given topic
    Returns: Dictionary with search results
    """
    print(f"\n Searching for papers on: '{topic}'")
    print(f"   Requesting {limit} papers from Semantic Scholar...")

    sch = setup_api_key()

    try:
        # Search for papers
        results = sch.search_paper(
            query=topic,
            limit=limit,
            fields=["paperId", "title", "abstract", "year", "authors",
                   "citationCount", "openAccessPdf", "url", "venue"]
        )

        papers = []
        for paper in results:
            paper_data = {
                "title": paper.title,
                "authors": [author['name'] for author in paper.authors] if paper.authors else [],
                "year": paper.year,
                "paperId": paper.paperId,
                "abstract": paper.abstract[:300] + "..." if paper.abstract else "No abstract available",
                "citationCount": paper.citationCount,
                "venue": paper.venue if hasattr(paper, 'venue') else None,
                "url": paper.url,
                "pdf_url": paper.openAccessPdf['url'] if paper.openAccessPdf else None,
                "has_pdf": bool(paper.openAccessPdf)
            }
            papers.append(paper_data)

        # Calculate statistics
        papers_with_pdf = sum(1 for p in papers if p["has_pdf"])

        print(f"Search complete!")
        print(f"   Total papers found: {len(papers)}")
        print(f"   Papers with PDF available: {papers_with_pdf}")

        return {
            "topic": topic,
            "search_timestamp": "timestamp_placeholder",
            "total_results": len(papers),
            "papers_with_pdf": papers_with_pdf,
            "papers": papers
        }

    except Exception as e:
        print(f" Error searching papers: {e}")
        return None

In [5]:
def save_search_results(data, filename=None):
    """
    Save search results to JSON file
    """
    if not filename:
        # Create filename from topic
        safe_topic = "".join(c for c in data["topic"] if c.isalnum() or c == " ").replace(" ", "_")
        filename = f"paper_search_results_{safe_topic}.json"

    os.makedirs("data/search_results", exist_ok=True)
    filepath = os.path.join("data/search_results", filename)

    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

    print(f" Search results saved to: {filepath}")
    return filepath

In [6]:
import pandas as pd

def display_results_table(data):
    """
    Convert search results into a clean, readable table using pandas.
    """
    if not data or "papers" not in data:
        print("No data to display in table")
        return

    table_data = []

    for p in data["papers"]:
        table_data.append({
            "Title": p["title"],
            "Authors": ", ".join(p["authors"][:3]) + ("..." if len(p["authors"]) > 3 else ""),
            "Year": p["year"],
            "Citations": p["citationCount"],
            "PDF": "Yes" if p["has_pdf"] else "No",
            "Venue": p["venue"]
        })

    df = pd.DataFrame(table_data)

    print("\n" + "="*80)
    print("TABLE VIEW OF RESULTS")
    print("="*80)
    display(df)

    return df


In [7]:
def main_search():
    """
    Main function for Module 1: Get topic and search for papers
    """
    print("\n" + "="*80)
    print("MODULE 1: TOPIC INPUT & PAPER SEARCH")
    print("="*80)

    # Get topic from user
    topic = input("\nEnter research topic: ").strip()
    if not topic:
        topic = "machine learning"  # Default topic

    # Search for papers
    results = search_papers(topic, limit=20)

    if results:
        # Save results
        save_path = save_search_results(results)

        # Display results
        display_results_table(results)

        print(f"\n Module 1 complete! Results saved to: {save_path}")

        return results, save_path
    else:
        print(" No results found. Please try a different topic.")
        return None, None

# Run Module 1 directly if needed
if __name__ == "__main__":
    main_search()


MODULE 1: TOPIC INPUT & PAPER SEARCH

Enter research topic: AI system to automatically review and summarize research papers

 Searching for papers on: 'AI system to automatically review and summarize research papers'
   Requesting 20 papers from Semantic Scholar...
Semantic Scholar initialized with API key
Search complete!
   Total papers found: 11
   Papers with PDF available: 11
 Search results saved to: data/search_results/paper_search_results_AI_system_to_automatically_review_and_summarize_research_papers.json

TABLE VIEW OF RESULTS


,Title,Authors,Year,Citations,PDF,Venue
0,Streamlining Academic Insights Gen AI in Resea...,"Priyadharshini P, Sahanna B, Sangeetha G...",2025,0,Yes,Pertanika Proceedings
1,AI Personal Study Buddy: A Web-Based Adaptive ...,"Shree shambhavi, Atharav Gadade, Ankita Birajd...",2025,0,Yes,INTERNATIONAL JOURNAL OF SCIENTIFIC RESEARCH I...
2,AI-Based Weapon Detection for Security Surveil...,"Thangavel Murugan, Nasurudeen Ahamed Noor Moha...",2025,0,Yes,Electronics
3,Automatic assessment of text-based responses i...,"Rujun Gao, H. Merzdorf, S. Anwar...",2023,91,Yes,Computers and Education: Artificial Intelligence
4,Editorial for Special Issue on Large-scale Pre...,"Jiying Wen, Zi-Hao Huang, Hanwang Zhang",2023,2,Yes,Machine Intelligence Research
5,Special issue on future hybrid artificial inte...,G. Ramírez-González,2021,0,Yes,Expert Syst. J. Knowl. Eng.
6,Special issue on intelligent computing methodo...,"Jinghua Zhao, Junyu Xuan",2021,0,Yes,Neural computing & applications (Print)
7,Tensorial modeling of dynamical systems for ga...,Fabio Cuzzolin,2010,0,Yes,
8,Advances and challenges in artificial intellig...,"Bing Li, Peng Yang, Yuankang Sun...",2024,11,Yes,Frontiers of Information Technology & Electron...
9,Natural Language Processing in Generating Indu...,"Izabela Rojek, Olga Małolepsza, Mirosław Kozie...",2025,0,Yes,Applied Sciences



 Module 1 complete! Results saved to: data/search_results/paper_search_results_AI_system_to_automatically_review_and_summarize_research_papers.json


In [8]:
!pip install ImageHash


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 9.3 MB/s eta 0:00:00


In [9]:
# removing the duplicates form the dataset
import os
import imagehash
from PIL import Image
from glob import glob

def remove_duplicate_images(folder_paths):
    """
    Remove duplicate images from given folders based on perceptual hash.
    Keeps the first image and deletes subsequent duplicates.
    """
    seen_hashes = {}
    removed = 0

    for folder in folder_paths:
        print(f"\nChecking folder: {folder}")

        image_files = glob(os.path.join(folder, "*"))

        for img_path in image_files:
            try:
                img = Image.open(img_path)

                # Compute perceptual hash
                img_hash = imagehash.phash(img)

                if img_hash in seen_hashes:
                    # Duplicate found → remove file
                    os.remove(img_path)
                    removed += 1
                    print(f"Removed duplicate: {img_path}")
                else:
                    seen_hashes[img_hash] = img_path

            except Exception as e:
                print(f"Error reading {img_path}: {e}")

    print(f"\nTotal duplicates removed: {removed}")


# Folders to check
folders = [
    "/content/dataset/Combined Dataset/Very_Mild_Demented",
    "/content/dataset/Combined Dataset/Mild_Demented",
    "/content/dataset/Combined Dataset/Moderate_Demented",
    "/content/dataset/Combined Dataset/Non_Demented"
]

# Call the function
remove_duplicate_images(folders)



Checking folder: /content/dataset/Combined Dataset/Very_Mild_Demented

Checking folder: /content/dataset/Combined Dataset/Mild_Demented

Checking folder: /content/dataset/Combined Dataset/Moderate_Demented

Checking folder: /content/dataset/Combined Dataset/Non_Demented

Total duplicates removed: 0
